# Transfer Learning

Most of the time you won't want to train a whole convolutional network yourself. Modern ConvNets training on huge datasets like ImageNet take weeks on multiple GPUs. Instead, most people use a pretrained network either as a fixed feature extractor, or as an initial network to fine tune. In this notebook, you'll be using VGGNet trained on the ImageNet dataset as a feature extractor. Below is a diagram of the VGGNet architecture.

VGGNet is great because it's simple and has great performance, coming in second in the ImageNet competition. The idea here is that we keep all the convolutional layers, but replace the final fully connected layers with our own classifier. This way we can use VGGNet as a feature extractor for our images then easily train a simple classifier on top of that. What we'll do is take the first fully connected layer with 4096 units, including thresholding with ReLUs. We can use those values as a code for each image, then build a classifier on top of those codes.

## Pretrained VGGNet

We'll be using a pretrained network from https://github.com/machrisaa/tensorflow-vgg.

This is a really nice implementation of VGGNet, quite easy to work with. The network has already been trained and the parameters are available from this link.

In [1]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm

In [3]:
vgg_dir = 'tensorflow_vgg/'
if not isdir(vgg_dir):
    raise Exception("VGG directory doesn't exists")

class DLProgress(tqdm):
    last_block = 0
    
    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num
    
if not isfile(vgg_dir + 'vgg15.npy'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='VGG16 Parameters') as pbar:
        urlretrieve('https://s3.amazonaws.com/content.udacity-data.com/nd101/vgg16.npy',
                    vgg_dir + 'vgg16.npy', pbar.hook)
else:
    print('Parameters file already exists')

VGG16 Parameters: 553MB [28:38, 322kB/s]                                                                               


In [4]:
import tarfile

dataset_folder_path = 'flower_photos'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile('flower_photos.tar.gz'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Flowers Dataset') as pbar:
        urlretrieve(
            'http://download.tensorflow.org/example_images/flower_photos.tgz',
            'flower_photos.tar.gz',
            pbar.hook)

if not isdir(dataset_folder_path):
    with tarfile.open('flower_photos.tar.gz') as tar:
        tar.extractall()
        tar.close()

Flowers Dataset: 229MB [02:23, 1.60MB/s]                                                                               


In [5]:
import os
import numpy as np
import tensorflow as tf

from tensorflow_vgg import vgg16
from tensorflow_vgg import utils

C:\Users\snettani\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
data_dir = 'flower_photos/'
contents = os.listdir(data_dir)
classes = [cls for cls in contents if os.path.isdir(data_dir+cls)]

Running the images through the VGG network in batches

In [10]:
batch_size = 10
codes_list = []
labels = []
batch = []

codes = None

with tf.Session() as sess:
    vgg = vgg16.Vgg16()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    with tf.name_scope('content_vgg'):
        vgg.build(input_)
    
    for cls in classes:
        print('starting {} images'.format(cls))
        class_path = data_dir + cls
        files = os.listdir(class_path)
        for i, file in enumerate(files, 1):
            #add images to the current batch
            #utils.load_image crops the images for us, from the center
            img = utils.load_image(os.path.join(class_path, file))
            batch.append(img.reshape((1, 224, 224, 3)))
            labels.append(cls)
            
            #running the batch through the network to get the codes
            if i%batch_size == 0 or i == len(files):
                images = np.concatenate(batch)
                feed_dict = {input_:images}
                codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)
                
                #Build an array of codes
                if codes is None:
                    codes = codes_batch
                else:
                    codes = np.concatenate((codes, codes_batch))
                
                #reset to start building the next bacth
                batch = []
                print('images processed'.format(i))

C:\Users\snettani\Documents\Personal\Github\Udacity_DeepLearning\TransferLearning\tensorflow_vgg\vgg16.npy
npy file loaded
build model started
build model finished: 4s
starting daisy images


C:\Users\snettani\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images processed
images process

In [11]:
#write codes to the file
with open('codes', 'w') as fp:
    codes.tofile(fp)

#write labels to the file
import csv
with open('labels', 'w') as fp:
    writer = csv.writer(fp, delimiter='\n')
    writer.writerow(labels)

## Building the classifier

Now that we have codes for all the images, we can build a simple classifier on top of them. The codes behave just like normal input into a simple neural network. 

In [12]:
#read codes and labels from the file
with open('codes') as fp:
    codes = np.fromfile(fp, dtype=np.float32)
    codes = codes.reshape((len(labels), -1))

    
with open('labels') as fp:
    reader = csv.reader(fp, delimiter='\n')
    labels = np.array([each for each in reader if len(each) > 0]).squeeze()

## Data prep

As usual, now we need to one-hot encode our labels and create validation/test sets. First up, creating our labels!

In [20]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(labels)
labels_vecs = lb.transform(labels)

Now you'll want to create your training, validation, and test sets. An important thing to note here is that our labels and data aren't randomized yet. We'll want to shuffle our data so the validation and test sets contain data from all classes. Otherwise, you could end up with testing sets that are all one class. Typically, you'll also want to make sure that each smaller set has the same the distribution of classes as it is for the whole data set. The easiest way to accomplish both these goals is to use StratifiedShuffleSplit from scikit-learn.

In [21]:
from sklearn.model_selection import StratifiedShuffleSplit

ss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
train_idx, val_idx = next(ss.split(codes, labels_vecs))
half_val_len = int(len(val_idx)/2)
val_idx, test_idx = val_idx[:half_val_len], val_idx[half_val_len:]

train_x, train_y = codes[train_idx], labels_vecs[train_idx]
val_x, val_y = codes[val_idx], labels_vecs[val_idx]
test_x, test_y = codes[test_idx], labels_vecs[test_idx]

In [22]:
print("Train shapes (x, y):", train_x.shape, train_y.shape)
print("Validation shapes (x, y):", val_x.shape, val_y.shape)
print("Test shapes (x, y):", test_x.shape, test_y.shape)

Train shapes (x, y): (2936, 4096) (2936, 5)
Validation shapes (x, y): (367, 4096) (367, 5)
Test shapes (x, y): (367, 4096) (367, 5)


## Classifier layers

Once you have the convolutional codes, you just need to build a classfier from some fully connected layers. You use the codes as the inputs and the image labels as targets. Otherwise the classifier is a typical neural network.